# Airline Complaint System - Complete Test Suite

This notebook demonstrates all features of the enhanced airline complaint system using TrustCall and LangGraph patterns.

## Prerequisites
- Make sure the API is running: `python main.py`
- API should be accessible at http://localhost:8002

In [ ]:
# Import required libraries
import requests
import json
import time
from IPython.display import display, HTML, Image
import pandas as pd

# Configuration
API_URL = "http://localhost:8002"

# Helper function for pretty printing
def print_response(response):
    """Pretty print API response"""
    if response.status_code == 200:
        data = response.json()
        print(json.dumps(data, indent=2))
        return data
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

## 1. System Health Check

In [ ]:
# Check if API is running
print("Checking API health...\n")
response = requests.get(f"{API_URL}/api/v2/health")
print_response(response)

# Get API info
print("\nAPI Information:")
response = requests.get(f"{API_URL}/")
print_response(response)

## 2. Clear Database (Fresh Start)

In [ ]:
# Clear all existing tickets
print("Clearing database...")
response = requests.delete(f"{API_URL}/api/v2/tickets/clear")
print_response(response)

## 3. Test Scenario 1: Complete Complaint (All Info at Once)

In [ ]:
print("SCENARIO 1: Complete complaint with all information\n")
print("="*60)

complaint = {
    "message": "I am Alice Johnson, alice@example.com, phone 555-0123. Yesterday on flight UA456 from NYC to LA, we had a 5 hour delay and then my luggage was lost! This ruined my business trip.",
    "thread_id": "complete_001"
}

print(f"Sending: {complaint['message']}\n")
response = requests.post(f"{API_URL}/api/v2/complaint", json=complaint)
data = print_response(response)

if data and data.get('ticket_id'):
    print(f"\n✓ Ticket created successfully: {data['ticket_id']}")
    print(f"Status: {data['status']}")

## 4. Test Scenario 2: Progressive Complaint Building

In [ ]:
print("SCENARIO 2: Progressive complaint building\n")
print("="*60)

thread_id = "progressive_001"
messages = [
    "I want to file a complaint about my recent flight",
    "It was flight BA789 from London to Paris",
    "We had a 3 hour delay and the service was terrible",
    "My name is Bob Smith",
    "My email is bob.smith@example.com"
]

for i, msg in enumerate(messages, 1):
    print(f"\nMessage {i}: '{msg}'")
    response = requests.post(
        f"{API_URL}/api/v2/complaint",
        json={"message": msg, "thread_id": thread_id}
    )
    data = response.json()
    
    if data.get('ticket_id'):
        print(f"✓ Ticket created: {data['ticket_id']}")
        break
    else:
        print(f"Status: {data['status']}")
        if data.get('missing_fields'):
            print(f"Missing: {', '.join(data['missing_fields'])}")

## 5. Test Scenario 3: Ticket Updates

In [ ]:
print("SCENARIO 3: Updating existing ticket\n")
print("="*60)

# First create a ticket
thread_id = "update_001"
initial = {
    "message": "I'm Carol White, carol@test.com. Lost luggage on flight LH123.",
    "thread_id": thread_id
}

print("Creating initial ticket...")
response = requests.post(f"{API_URL}/api/v2/complaint", json=initial)
data = response.json()
ticket_id = data.get('ticket_id')
print(f"Ticket created: {ticket_id}\n")

# Send updates
updates = [
    "My phone number is 555-9999",
    "The luggage tag number is LH789456",
    "This is urgent - the luggage contains medication"
]

for update in updates:
    print(f"Sending update: '{update}'")
    response = requests.post(
        f"{API_URL}/api/v2/complaint",
        json={"message": update, "thread_id": thread_id}
    )
    data = response.json()
    print(f"Status: {data['status']}\n")

## 6. Test API Call Optimizations

In [ ]:
print("TESTING API CALL OPTIMIZATIONS\n")
print("="*60)

print("\n1. Trivial messages (no LLM call):")
trivial_messages = ['ok', 'thanks', 'yes']
for msg in trivial_messages:
    response = requests.post(f"{API_URL}/api/v2/complaint", json={"message": msg})
    data = response.json()
    print(f"  '{msg}' -> {data['status']} (NO LLM CALL)")

print("\n2. Duplicate detection:")
thread_id = "dup_test"
duplicate_msg = "My flight was delayed"
for i in range(2):
    response = requests.post(
        f"{API_URL}/api/v2/complaint",
        json={"message": duplicate_msg, "thread_id": thread_id}
    )
    data = response.json()
    if i == 0:
        print(f"  First: '{duplicate_msg}' -> {data['status']} (LLM CALLED)")
    else:
        print(f"  Duplicate: '{duplicate_msg}' -> {data['status']} (NO LLM CALL)")

## 7. Fetch and Display All Tickets

In [ ]:
# Get all tickets
print("Fetching all tickets...\n")
response = requests.get(f"{API_URL}/api/v2/tickets")
data = response.json()

print(f"Total tickets: {data['count']}\n")

if data['tickets']:
    # Convert to DataFrame for nice display
    df = pd.DataFrame(data['tickets'])
    columns_to_show = ['ticket_id', 'passenger_name', 'passenger_email', 'flight_number', 
                      'category', 'priority', 'status', 'created_at']
    df_display = df[columns_to_show].head(10)
    display(df_display)
else:
    print("No tickets found")

## 8. Test Get Ticket by ID

In [ ]:
# Get tickets first
response = requests.get(f"{API_URL}/api/v2/tickets")
data = response.json()

if data['tickets']:
    # Get the first ticket
    ticket_id = data['tickets'][0]['ticket_id']
    
    print(f"Fetching ticket: {ticket_id}\n")
    response = requests.get(f"{API_URL}/api/v2/tickets/{ticket_id}")
    
    if response.status_code == 200:
        ticket = response.json()['ticket']
        print("Ticket Details:")
        print("="*40)
        for key, value in ticket.items():
            if value:
                print(f"{key:20}: {value}")
    else:
        print(f"Error fetching ticket: {response.status_code}")
else:
    print("No tickets available to fetch")

## 9. Understanding the original_complaint Field

In [ ]:
print("UNDERSTANDING COMPLAINT SUMMARIZATION\n")
print("="*60)

# Clear database for clean test
requests.delete(f"{API_URL}/api/v2/tickets/clear")

# Test with verbose complaint
verbose_complaint = {
    "message": """I am extremely frustrated! My name is David Brown, david@test.com. 
    So yesterday I was supposed to fly on AA888 from Chicago to Miami for my daughter's wedding. 
    First, they delayed the flight by 2 hours with no explanation. Then when we finally boarded, 
    we sat on the tarmac for another hour. To make matters worse, when I finally arrived, 
    my luggage was nowhere to be found! The staff was unhelpful and rude. 
    This has been the worst travel experience of my life!""",
    "thread_id": "summary_test"
}

print("Original message (verbose):")
print(verbose_complaint['message'][:200] + "...\n")

response = requests.post(f"{API_URL}/api/v2/complaint", json=verbose_complaint)
data = response.json()

if data.get('ticket_id'):
    # Fetch the ticket to see the summarized complaint
    ticket_response = requests.get(f"{API_URL}/api/v2/tickets/{data['ticket_id']}")
    ticket = ticket_response.json()['ticket']
    
    print("LLM-generated summary stored in database:")
    print("-"*40)
    print(ticket['original_complaint'])
    print("-"*40)
    print("\n✓ The LLM intelligently summarized the key issues!")

## 10. View LangGraph Workflow

In [ ]:
# Display link to graph visualization
print("LangGraph Workflow Visualization\n")
print("="*60)
print("\nThe LangGraph workflow can be viewed at:")
print(f"http://localhost:8002/api/v2/graph")
print("\nWorkflow nodes:")
print("  1. CLASSIFY - Determine if message is a complaint")
print("  2. EXTRACT - Extract passenger information")
print("  3. DECIDE_ACTION - Route to appropriate action")
print("  4. ANALYZE - Categorize and prioritize complaint")
print("  5. EXECUTE_ACTION - Create/update ticket")
print("  6. RESPOND - Generate customer response")

# Try to display the graph inline (if running in Jupyter)
try:
    from IPython.display import IFrame
    display(IFrame(src="http://localhost:8002/api/v2/graph", width=1000, height=600))
except:
    print("\nOpen the URL above in your browser to see the graph visualization")

## 11. Performance Summary

In [ ]:
# Get final statistics
response = requests.get(f"{API_URL}/api/v2/tickets")
data = response.json()

print("SYSTEM PERFORMANCE SUMMARY\n")
print("="*60)
print(f"\nTotal tickets created: {data['count']}")

if data['tickets']:
    df = pd.DataFrame(data['tickets'])
    
    # Category distribution
    print("\nTickets by Category:")
    if 'category' in df.columns:
        print(df['category'].value_counts().to_string())
    
    # Priority distribution
    print("\nTickets by Priority:")
    if 'priority' in df.columns:
        print(df['priority'].value_counts().to_string())
    
    # Status distribution
    print("\nTickets by Status:")
    if 'status' in df.columns:
        print(df['status'].value_counts().to_string())

print("\n" + "="*60)
print("KEY ACHIEVEMENTS:")
print("  ✓ TrustCall pattern with structured outputs")
print("  ✓ Single LLM call per request (optimized)")
print("  ✓ Progressive information gathering")
print("  ✓ Intelligent complaint summarization")
print("  ✓ API call optimizations for trivial messages")
print("  ✓ PostgreSQL persistence")
print("  ✓ Clean, maintainable codebase")

## 12. Cleanup (Optional)

In [ ]:
# Uncomment to clear all tickets
# print("Clearing all tickets...")
# response = requests.delete(f"{API_URL}/api/v2/tickets/clear")
# print_response(response)